# 23. GRADUATE ADMISSION: MODEL TRAINING 1
---

## 1. Presenting Our Objectives

- We will training 4 different models to find the best:
    - OLS Linear Regression
    - ElasticNet Linear Regression
    - K-Nearest Neighbors Regression
    - Random Forest Regression
    
- We will build prediction models in 2 different ways:
    - Manual: design one feature that will be used for a univariate model
    - Traditional: design models as it is usually done (train all features and see the results)
    
- We will work on 3 different variations of our data
    - Data 1: No changes to the dataset
    - Data 2: Continuous features are discretized (binned)
    - Data 3: Remove target outliers and 
        - bin continous features if `Data 2` were better than `Data 1`, 
        - no binning if `Data 2` were worse than `Data 1`
        
- We could end up training 3x2x4=`24 variations of models`

## 2. Introducing the Data
#### `No changes to the dataset`